In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split

In [2]:
base_dir = os.path.abspath('datafiles')
categories = ['Cheetah', 'Crocodile', 'Elephants', 'Tiger']
data = []
for category in categories:
    category_path = os.path.join(base_dir, category)
    if os.path.isdir(category_path):
        for file_name in os.listdir(category_path):
            # print(file_name)
            data.append([os.path.join(category_path, file_name), category])

df = pd.DataFrame(data, columns=['file_path', 'label'])
# print(df)

In [3]:
df['label'] = df['label'].astype('category').cat.codes
train_df, test_df = train_test_split(df, test_size=0.1, random_state=42)
test_df

,file_path,label
1448,d:\projects-activities\Machine-Learning-LS-24\...,3
168,d:\projects-activities\Machine-Learning-LS-24\...,0
220,d:\projects-activities\Machine-Learning-LS-24\...,0
1643,d:\projects-activities\Machine-Learning-LS-24\...,3
344,d:\projects-activities\Machine-Learning-LS-24\...,0
...,...,...
188,d:\projects-activities\Machine-Learning-LS-24\...,0
350,d:\projects-activities\Machine-Learning-LS-24\...,0
1460,d:\projects-activities\Machine-Learning-LS-24\...,3
944,d:\projects-activities\Machine-Learning-LS-24\...,1


In [4]:
def load_image(file_path, label):
    img = tf.io.read_file(file_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, [64, 64])
    img = img / 255.0
    return img, label

def augment_image(img, label):
    img = tf.image.random_flip_left_right(img)
    img = tf.image.random_flip_up_down(img)
    img = tf.image.random_brightness(img, max_delta=0.2)
    img = tf.image.random_contrast(img, lower=0.8, upper=1.2)
    return img, label

train_dataset = tf.data.Dataset.from_tensor_slices((train_df['file_path'].values, train_df['label'].values))
train_dataset = train_dataset.map(load_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)
train_dataset = train_dataset.map(augment_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)
train_dataset = train_dataset.shuffle(buffer_size=10000).batch(16).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

test_dataset = tf.data.Dataset.from_tensor_slices((test_df['file_path'].values, test_df['label'].values))
test_dataset = test_dataset.map(load_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)
test_dataset = test_dataset.batch(16).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

In [5]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Dropout(0.25),

    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Dropout(0.25),

    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Dropout(0.25),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(len(categories), activation='softmax')
])


optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

c:\Users\aditi\miniconda3\envs\project\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 62, 62, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 62, 62, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 31, 31, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 31, 31, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 29, 29, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 29, 29, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 12, 12, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 12, 12, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 6, 6, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 6, 6, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 4608)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 512)            │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4)              │         2,052 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,458,052 (9.38 MB)

 Trainable params: 2,456,580 (9.37 MB)

 Non-trainable params: 1,472 (5.75 KB)

In [6]:
history = model.fit(
    train_dataset,
    epochs=150,
    validation_data=test_dataset
)

Epoch 1/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 16s 95ms/step - accuracy: 0.3073 - loss: 2.1455 - val_accuracy: 0.2754 - val_loss: 1.4175
Epoch 2/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 10s 80ms/step - accuracy: 0.4029 - loss: 1.9004 - val_accuracy: 0.2754 - val_loss: 1.6701
Epoch 3/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 10s 80ms/step - accuracy: 0.4198 - loss: 1.7177 - val_accuracy: 0.2874 - val_loss: 1.7720
Epoch 4/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 10s 80ms/step - accuracy: 0.4555 - loss: 1.5303 - val_accuracy: 0.3054 - val_loss: 2.1584
Epoch 5/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 10s 80ms/step - accuracy: 0.4610 - loss: 1.5182 - val_accuracy: 0.3473 - val_loss: 1.6121
Epoch 6/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 10s 80ms/step - accuracy: 0.5015 - loss: 1.4768 - val_accuracy: 0.4072 - val_loss: 1.7374
Epoch 7/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 10s 82ms/step - accuracy: 0.4987 - loss: 1.4153 - val_accuracy: 0.4731 - val_loss: 1.4639
Epoch 8/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 11s 80ms/step - accuracy: 0.4843 - loss: 1.4012 - val_accu

In [7]:
def test_model(model, test_dataset):
    test_loss, test_accuracy = model.evaluate(test_dataset)
    test_accuracy_percentage = test_accuracy * 100
    
    print(f'Test Accuracy: {test_accuracy_percentage:.2f}%')
    print(f'Test Loss: {test_loss}')


test_model(model, test_dataset)

11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.8441 - loss: 0.6625
Test Accuracy: 82.04%
Test Loss: 0.7966597080230713
